# Deep Dive into LangChain
## LLMs, Prompt Templates, Caching, Streaming, Chains

This notebook uses the latest versions of the OpenAI and LangChain libraries.

In [ ]:
!pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.

Download [requirements.txt](https://drive.google.com/file/d/1UpURYL9kqjXfe9J8o-_Dq5KJTbQpzMef/view?usp=sharing)

In [ ]:
!pip install --upgrade -q langchain

In [ ]:
!pip install --upgrade -q openai

In [ ]:
!pip show openai

Name: openai
Version: 1.23.6
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai


In [ ]:
!pip show langchain

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental


### Python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

print(os.environ.get('OPENAI_API_KEY'))

## Chat Models: GPT-3.5 Turbo and GPT-4

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and instead, phenomena such as superposition and entanglement occur.


In [ ]:
# help(ChatOpenAI)  # see the llm constructor arguments with its defaults

In [ ]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in Telugu.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

క్వాంటం యంత్రశాస్త్రం ఒక శాస్త్రం యొక్క వ్యవహారాత్మక సిద్ధాంతాన్ని వివరిస్తుంది, వాంతి విజ్ఞాన నియమాలను మీరు వివరిస్తున్నారు.


## Caching LLM Responses

### 1. In-Memory Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [ ]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [ ]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 30.4 ms, sys: 4.72 ms, total: 35.1 ms
Wall time: 765 ms


'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

In [ ]:
%%time
llm.invoke(prompt)

CPU times: user 582 µs, sys: 0 ns, total: 582 µs
Wall time: 590 µs


'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

### 2. SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path="/content/.langchain.db"))

In [ ]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: user 34.6 ms, sys: 2.99 ms, total: 37.6 ms
Wall time: 586 ms


'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

In [ ]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: user 17.6 ms, sys: 4.94 ms, total: 22.6 ms
Wall time: 26.2 ms


'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

## LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the pale moonlight
A raven flies, its wings so black and bright
It calls out to the moon, a haunting melody
A creature of the night, wild and free

Chorus:
Moon and raven, dancing in the sky
Their spirits intertwined, never asking why
They roam the night, together as one
Moon and raven, under the midnight sun

Verse 2:
The moon shines down, a beacon in the dark
Guiding the raven on its journey, leaving its mark
Through forests and valleys, over mountains high
The moon and raven, never say goodbye

Chorus:
Moon and raven, dancing in the sky
Their spirits intertwined, never asking why
They roam the night, together as one
Moon and raven, under the midnight sun

Bridge:
The moon whispers secrets, the raven knows them all
They share a bond, unbreakable and tall
Through the storms and the shadows, they will always be
Moon and raven, forever wild and free

Chorus:
Moon and raven, dancing in the sky
Their spirits intertwined, never asking why
They roam the 

In [ ]:
print(llm.invoke(prompt))

content='Verse 1:\nIn the dead of night, under the pale moonlight\nA raven flies, its wings so black and bright\nIt calls out to the moon, a haunting melody\nA creature of the night, wild and free\n\nChorus:\nMoon and raven, dancing in the sky\nTheir spirits intertwined, never asking why\nThey roam the night, together as one\nMoon and raven, under the midnight sun\n\nVerse 2:\nThe moon shines down, a beacon in the dark\nGuiding the raven on its journey, leaving its mark\nThrough forests and valleys, over mountains high\nThe moon and raven, never say goodbye\n\nChorus:\nMoon and raven, dancing in the sky\nTheir spirits intertwined, never asking why\nThey roam the night, together as one\nMoon and raven, under the midnight sun\n\nBridge:\nThe moon whispers secrets, the raven knows them all\nThey share a bond, unbreakable and tall\nThrough the storms and the shadows, they will always be\nMoon and raven, forever wild and free\n\nChorus:\nMoon and raven, dancing in the sky\nTheir spirits int

In [ ]:
c = "Its piercing eyes, a story yet untold,"
print(c,end='.')

Its piercing eyes, a story yet untold,.

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Beneath the midnight sky
The moon shines bright up high
A raven flies by
Its silhouette against the night

Chorus:
Oh, moon and raven
Dancing in the dark
A mysterious connection
Leaving its mark

Verse 2:
The raven calls out
A haunting melody
As the moon whispers secrets
To the creatures of the night

Chorus:
Oh, moon and raven
Guiding us through the night
A symbol of mystery
In the pale moonlight

Bridge:
The moon's silver glow
Reflecting in the raven's eye
A bond between two worlds
That will never die

Chorus:
Oh, moon and raven
Forever intertwined
In the endless cycle
Of the night's design

Outro:
As the raven takes flight
And the moon begins to fade
Their connection remains
In the darkness they've made.

## PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='telugu')
prompt  # Returns the generated prompt


'You are an experienced virologist.\nWrite a few sentences about the following virus "hiv" in telugu.'

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV వైరస్ ఒక మానవ ఇమ్యూనోడీఫీషీన్సీ వైరస్ (HIV) మరియు ఏడు విధానాలు ఉంటాయి. ఇది మానవ శరీరంలో కనిపించే మరియు మన ఇమ్యూనోసిస్టం నిలిచిపోవడానికి కారణం ఉంటుంది. ఈ వైరస్ మానవ జీవితాన్ని ప్రభావితం చేస్తుంది మరియు ఏడు విధానాలు ఉంటాయి.


## ChatPromptTemplates

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "name": "China",
            "population": "1,439,323,776"
        },
        {
            "rank": 2,
            "name": "India",
            "population": "1,380,004,385"
        },
        {
            "rank": 3,
            "name": "United States",
            "population": "331,002,651"
        },
        {
            "rank": 4,
            "name": "Indonesia",
            "population": "273,523,615"
        },
        {
            "rank": 5,
            "name": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in Spanish.

> Finished chain.


In [ ]:
print(output)

{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV) es un virus de doble cadena de ADN que pertenece a la familia Herpesviridae. Hay dos tipos principales de HSV: HSV-1, que generalmente causa herpes labial en la boca, y HSV-2, que generalmente causa herpes genital. Ambos tipos de virus son altamente contagiosos y pueden causar brotes recurrentes de lesiones en la piel y membranas mucosas. Actualmente no existe una cura para el HSV, pero los medicamentos antivirales pueden ayudar a reducir la frecuencia y gravedad de los brotes.'}


In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])

Enter Country: india


> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of india?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of India is New Delhi.

Top 3 places to visit in New Delhi:
- Red Fort
- India Gate
- Qutub Minar


## Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is a Python function that implements linear regression:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    numerator = 0
    denominator = 0
    for i in range(n):
        numerator += (x[i] - x_mean) * (y[i] - y_mean)
        denominator += (x[i] - x_mean) ** 2

    slope = numerator / denominator
    intercept = y_mean - slope * x_mean

    return slope, intercept

# Example usage
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])

slope, intercept = linear_regression(x, y)
print(f"Slope: {slope}, Intercept: {intercept}")
```

This function takes two arrays `x` and `y` as input, where `x` is the independent variable and `y` is the dependent variable. It calculates the slope and intercept of the linear regression line that best fits the data points `(x, y)`. The function then returns the slope and intercept values.

You can use this function 

In [ ]:
print(output['output'])

The Python function `linear_regression` is an implementation of simple linear regression, which is a fundamental statistical approach used to model the relationship between a single independent variable (denoted as `x`) and a dependent variable (denoted as `y`). The main goal of this function is to find the best-fitting straight line (also known as the regression line) that describes this relationship. The equation of this line is `y = beta1 * x + beta0`, where `beta1` is the slope of the line, and `beta0` is the y-intercept.

The implementation details of the `linear_regression` function are explained below:

1. **Imports and Dependencies**: The function uses NumPy (imported as `np`), a popular library for numerical computing in Python. NumPy provides efficient ways to handle arrays and mathematical operations, making it an ideal choice for implementing linear regression.

2. **Parameters**: The function accepts two parameters, `x` and `y`, which are NumPy arrays of the same length. T

In [ ]:
pip install -q langchain_experimental

In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python for both the factorial and square root calculations.
Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [ ]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [ ]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [ ]:
print(response['output'])


146306.05007233328
